In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
VSDMPL_DATA = "" # path to parquet

In [ ]:
df = pd.read_parquet(VSDMPL_DATA)
df

In [ ]:
# ultimate_hosts = df[df['upid'] == -1]

In [ ]:
########################## TO DO: FIND & APPLY MEANINGFUL MAX CUTOFF APPROXIMATION -- APPLY MAGNITUDE LIMITATIONS OF SAGA SURVEY & __ DISTANCE? #######################
# Filter by most massive
subs = df[df['upid'] != -1][df['mvir'] >= 1.0e+10]
subs.sort_values(by=['pid', 'id'])

<h2>Plot & Plot data</h2>

In [ ]:
subs_test = subs.groupby(['pid']).filter(lambda x: len(x) > 2)

# add dummy rows to put parent data in graph - just easy method for graph grouping
test_parents = subs_test['pid'].unique()
subs_test_with_hosts = subs_test.copy()

for parent_id in test_parents:
    row = df[df['id'] == parent_id].copy()
    row['pid'] = row['id']
    subs_test_with_hosts = pd.concat([subs_test_with_hosts, row])
    
subs_test_with_hosts.sort_values(by=['id'])
subs_test_with_hosts.head(40)

In [ ]:
g = sns.FacetGrid(subs_test_with_hosts, col="pid", col_wrap=8, height=2, sharex=False, sharey=False, palette="viridis", hue="mvir")
g.map(sns.scatterplot, "x", "y")
g.map(sns.scatterplot, )

<h2>Ellipticity calculations</h2>

In [ ]:
data = subs.groupby(['pid']).filter(lambda x: len(x) > 2)

In [ ]:
parent_ids = data['pid'].unique()
ellipticity_df = pd.DataFrame({'pid': parent_ids}, columns=['pid', 'Q_xx', 'Q_yy', 'Q_xy'])

<h4>Quadrupole moments</h4>

In [ ]:
##################### TO DO: CHECK PRECISION OF SQUARES ####################
for parent_id in parent_ids:
    satellite_data = data[data['pid'] == parent_id].copy()
    parent_data = df[df['id'] == parent_id]
    
    if parent_data.size != 0:
        # calculate x_i - x_c
        satellite_data['x_diff'] = satellite_data['x'].subtract(parent_data['x'].loc[parent_data.index[0]])
        satellite_data['y_diff'] = satellite_data['y'].subtract(parent_data['y'].loc[parent_data.index[0]])
        
        # square 
        satellite_data['x_diff_sq'] = satellite_data['x_diff']**2
        satellite_data['y_diff_sq'] = satellite_data['y_diff']**2
        satellite_data['x_diff_y_diff'] = satellite_data['y_diff']*satellite_data['x_diff']
        
        ellipticity_df.loc[ellipticity_df.pid == parent_id, 'Q_xx'] = satellite_data['x_diff_sq'].mean()
        ellipticity_df.loc[ellipticity_df.pid == parent_id, 'Q_yy'] = satellite_data['y_diff_sq'].mean()
        ellipticity_df.loc[ellipticity_df.pid == parent_id, 'Q_xy'] = satellite_data['x_diff_y_diff'].mean()
    else:
        print("No parent data for parent_id: {}".format(parent_id))

<h4>Ellipticity components</h4>

In [ ]:
def e_1 (Q_xx, Q_yy, Q_xy):
    return (( Q_xx - Q_yy )/(Q_xx + Q_yy + 2*(Q_xx*Q_yy-Q_xy**2)**(1/2)))
def e_2 (Q_xx, Q_yy, Q_xy):
    return (2*Q_xy/(Q_xx + Q_yy + 2*(Q_xx*Q_yy-Q_xy**2)**(1/2)))
def e_3 (e_1, e_2):
    return ( e_1**2 + e_2**2 )**(1/2)


ellipticity_df['e_1'] = e_1(ellipticity_df['Q_xx'], ellipticity_df['Q_yy'], ellipticity_df['Q_xy'])
ellipticity_df['e_2'] = e_2(ellipticity_df['Q_xx'], ellipticity_df['Q_yy'], ellipticity_df['Q_xy'])
ellipticity_df['e_3'] = e_3(ellipticity_df['e_1'], ellipticity_df['e_2'])

ellipticity_df

In [ ]:
sns.histplot(ellipticity_df["e_3"], bins=8)

In [ ]:
# check about the high ellipticity bois
high_ellipticity_halo_ids = ellipticity_df[ellipticity_df['e_3'] >= 0.9]['pid'].unique()
high_ellipticity_data = data[data['pid'].isin(high_ellipticity_halo_ids)]

high_ellipticity_halos = high_ellipticity_data.groupby(['pid'])['id'].count()
average_count_high_ellipticity = high_ellipticity_halos.mean()
print("Average number of satellites per halo for the high-eccentricity data: {}".format(average_count_high_ellipticity))

all_halos = data.groupby(['pid'])['id'].count()
average_count_all = all_halos.mean()
print("Average number of satellites per halo for entire data set: {}".format(average_count_all))
